# Clicar em arquivo e salvar uma cópia no Drive
## Fonte/créditos: https://github.com/theAIGuysCode/yolov4-deepsort
## Github: https://github.com/altanizio/Pedestre-Atraso-Brecha_aceita-yolov4-deepsort

# ⚙ Pré-processamento

O vídeo de análise deve estar no Google Drive e estar em .mp4.

O código abaixo seta o diretório no drive e apresenta funções pra converter 📼 e cortar ✂ o vídeo. 

Os scripts estão em /scripts_pre-processamento no github

## Definição da área de conflito

Primeiro é nescessário executar o area_conflito.py para definir a área de conflito, pois o colab não permite a interação no output

Execute o script. Escolha o Vídeo. Clique em quatro dos veritices de interesse, comece pelo sentido de travessia dos pedestres. Aperte esc duas vezes. O último vetor do output é a definição da área de conflito.

<p align="left"><img src="https://raw.githubusercontent.com/altanizio/Pedestre-Atraso-Brecha_aceita-yolov4-deepsort/master/data/helpers/ex_cut.gif"\></p>



## Pastas e arquivos

### Passos:

Definição da pasta de trabalho. (rode sempre que iniciar)

Clonar o diretório do github com os códigos. (salve no seu drive para não precisar baixar sempre)

Baixar os pesos do modelo pré-treinado. (o mesmo)

In [ ]:
import os 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Montar o drive no colab, se precisar - precisa dessa pasta YOLO no diretório raiz
if os.path.isdir('/content/drive'):
  %cd drive/MyDrive/YOLO
else:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd drive/MyDrive/YOLO

In [ ]:
#if not os.path.isdir('core'):
#  print('a')

In [ ]:
# clone repository for deepsort with yolov4
#The following cell will clone the yolov4-deepsort repository from github, to enable the rest of the tutorial and grab the code.
#if not os.path.isdir('core'):
#  !git clone https://github.com/theAIGuysCode/yolov4-deepsort
#  %cd yolov4-deepsort/

In [ ]:
# download yolov4 model weights to data folder
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights -P data/

In [ ]:
# Convert darknet weights to tensorflow model
#!python save_model.py --model yolov4

In [ ]:
##!pip install -r requirements-gpu.txt

## Execute os scripts converter.cmd para converter os vídeos para uma condição adequada (mps, fps, etc.)

## Execute cut_videos.py para cortar os trechos para a análise.

Obs.: edite os códigos para adequar com os nomes e objetivos. É possível rodar os códigos aqui, entretanto é melhor rodar localmente esses scripts para melhor organização.

In [ ]:
# Cortar uma parte do vídeo específica
#from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
#ffmpeg_extract_subclip("./data/video/"+ nome_arquivo_video +".avi", 180, 240, targetname="./data/video/"+ nome_arquivo_video +".avi")

In [ ]:
# Converter o vídeo para mp4 -> nescessário para evitar problemas com o fps
#%cd ./data/video/
#!ffmpeg -y -loglevel panic -i domManuel.avi domManuel.mp4
#%cd ..
#%cd ..

In [ ]:
# Converter o vídeo para mp4 -> nescessário para evitar problemas com o fps
#%cd ./data/video/
#!ffmpeg -y -loglevel panic -i avAbolicao.avi -filter:v fps=10 avAbolicao.mp4
#%cd ..
#%cd ..

# Modelo de detecção

## Variáveis de entrada do modelo

In [ ]:
cont_line = [[1032, 596, 1099, 403], [1099, 403, 359, 273], [359, 273, 186, 433], [186, 433, 1032, 596]]  # Retângulo da área de conflito de interesse // começa pelo pedestre (primeiro dois pontos) //

nome_arquivo_video = 'avUniversidade_150_180'

if not os.path.exists('./outputs/'+ nome_arquivo_video):
    os.makedirs('./outputs/'+ nome_arquivo_video)
text_file = open('./outputs/'+ nome_arquivo_video +'/otm.txt', "w+")
text_file.write("cont_line = %s" % cont_line)
text_file.close()

# Fonte: https://github.com/theAIGuysCode/yolov4-deepsort
framework = 'tf' # tf, tflite, trt
weights = './checkpoints/yolov4-416' # path to weights file
size = 416 # resize images to
tiny = False #yolo or yolo-tiny
model = 'yolov4' # yolov3 or yolov4
video = "./data/video/"+ nome_arquivo_video +".mp4" # path to input video 
output = './outputs/'+ nome_arquivo_video +'/tracker.avi' # path to output video
output_format = 'XVID' # codec used in VideoWriter when saving video to file
iou = 0.45 # iou threshold
score = 0.5 # score threshold
dont_show = True
info = True # show detailed info of tracked objects
saida = True

#x_r = 20# tamanho do eixo x real em metros
#y_r = 9.2 # tamanho do eixo y real em metros

## Bibliotecas necessárias

In [ ]:
import os
# Fonte: https://github.com/theAIGuysCode/yolov4-deepsort
# comment out below line to enable tensorflow logging outputs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import time
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
import core.utils as utils
from core.yolov4 import filter_boxes
from tensorflow.python.saved_model import tag_constants
from core.config import cfg
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
# deep sort imports
from deep_sort import preprocessing, nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools import generate_detections as gdet
from collections import defaultdict
from tqdm import tqdm

# Manipulação dos dados
import pandas as pd

# Não mostrar avisos
import warnings
warnings.filterwarnings('ignore')

## Funções para verificar travessias

In [1]:
def centroid_box(box): #Retorna o centróide
  return [int(abs(box[0] + box[2])/2), int(abs(box[1] + box[3])/2)]

def dominio(ponto,linha): # Retorna se o ponto esta dentro da área de detecção - necessário para não contar pedestres ou carros que não estão na área de conflito de interesse
    if abs(linha[1]-linha[3]) > abs(linha[0]-linha[2]):
        if (ponto[1] - linha[1]) * (linha[3] - ponto[1]) >= -0.2:
            return True
        else:
            return False
    else:
        if (ponto[0] - linha[0]) * (linha[2] - ponto[0]) >= -0.2:
            return True
        else:
            return False  


def passou_da_linha(ponto,linha): # Verifica se o ponto esta acima ou abaixo de uma linha - com isso é possível verificar se o pedestre entrou na área de conflito
  #y = ax + b
  #try:  
    a = (linha[1]-linha[3])/(linha[0]-linha[2])
    b = linha[1] - a*linha[0]

    if ponto[1]<(a*ponto[0] + b):
        return 'acima'
    else:
        return 'abaixo'
  #except:
  #      return 'error'

def passou_da_linha_reta_horizontal(ponto,linha): # mesmo do anterior, mas de forma mais simples
    if ponto[1]>linha:
        return 'abaixo'
    else:
        return 'acima'

## Parâmetros

In [ ]:
# Fonte: https://github.com/theAIGuysCode/yolov4-deepsort
# Definition of the parameters
max_cosine_distance = 0.4
nn_budget = None
nms_max_overlap = 1.0

# initialize deep sort
model_filename = 'model_data/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
# calculate cosine distance metric
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
# initialize tracker
tracker = Tracker(metric)

# load configuration for object detector
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
#STRIDES, ANCHORS, NUM_CLASS, XYSCALE = utils.load_config(FLAGS)
input_size = size
video_path = video

#conflict_area = FLAGS.conflict_area
#print(conflict_area)
conflict_area = cont_line

# load tflite model if flag is set
if framework == 'tflite':
    interpreter = tf.lite.Interpreter(model_path=weights)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(input_details)
    print(output_details)
# otherwise load standard tensorflow saved model
else:
    saved_model_loaded = tf.saved_model.load(weights, tags=[tag_constants.SERVING])
    infer = saved_model_loaded.signatures['serving_default']

# begin video capture
try:
    vid = cv2.VideoCapture(int(video_path))
except:
    vid = cv2.VideoCapture(video_path)

out = None

# get video ready to save locally if flag is set
if output:
    # by default VideoCapture returns float instead of int
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    codec = cv2.VideoWriter_fourcc(*output_format)
    out = cv2.VideoWriter(output, codec, fps, (width, height))
    
session.close()

## Variáveis para coletar os dados desejados

In [ ]:
frame_num = 0 # número do frame
count_person = 0 # número de pedestres contados
id_track_list = [] # lista dos pedestres detectados
check_count = {} # aux para a contagem
check_count_aux = {} # aux para a contagem
check_count_2 = {} # aux para a contagem
check_count_2_aux = {} # aux para a contagem
time_person = {} # tempo que atravessou
time_person_aux = {} # aux para calcular o tempo que atravessou
x_person_inicial = {} # coordenadas inicias
y_person_inicial = {} # coordenadas inicias
x_person_final = {} # coordenadas finais
y_person_final = {} # coordenadas finais


id_track_list_2 = [] # lista de rastreamento de outros objetos (carros,...)
inicio_linha1 = {}
inicio_linha2 = {}
final_linha1 = {}
final_linha2 = {}

# similar ao pedestre
count_car = 0
check_count_car = {}
check_count_car_aux = {}
check_count_car_2 = {}
check_count_car_2_aux = {}
time_car = {}
time_car_aux = {}
x_car_inicial = {}
y_car_inicial = {}
x_car_final = {}
y_car_final = {}

car_class = {} # classe do veic

In [ ]:
class_names = utils.read_class_names(cfg.YOLO.CLASSES)

# by default allow all classes in .names file
allowed_classes = list(class_names.values())
allowed_classes # classes de detecção suportadas

## Detecção no vídeo

In [ ]:
# Fonte: https://github.com/theAIGuysCode/yolov4-deepsort
cont = tqdm(total = int(vid.get(cv2.CAP_PROP_FRAME_COUNT)), position=0, leave=True)
cont.set_description("Frames")
# while video is running
while True:
    return_value, frame = vid.read()
    if return_value:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame)
    else:
        #print('Video has ended or failed, try a different video format!')
        break
    frame_num +=1
    #print('Frame #: ', frame_num)
    #print('Segundos: ', frame_num/int(vid.get(cv2.CAP_PROP_FPS)))
    frame_size = frame.shape[:2]
    image_data = cv2.resize(frame, (input_size, input_size))
    image_data = image_data / 255.
    image_data = image_data[np.newaxis, ...].astype(np.float32)
    start_time = time.time()

    # run detections on tflite if flag is set
    if framework == 'tflite':
        interpreter.set_tensor(input_details[0]['index'], image_data)
        interpreter.invoke()
        pred = [interpreter.get_tensor(output_details[i]['index']) for i in range(len(output_details))]
        # run detections using yolov3 if flag is set
        if model == 'yolov3' and tiny == True:
            boxes, pred_conf = filter_boxes(pred[1], pred[0], score_threshold=0.25,
                                            input_shape=tf.constant([input_size, input_size]))
        else:
            boxes, pred_conf = filter_boxes(pred[0], pred[1], score_threshold=0.25,
                                            input_shape=tf.constant([input_size, input_size]))
    else:
        batch_data = tf.constant(image_data)
        pred_bbox = infer(batch_data)
        for key, value in pred_bbox.items():
            boxes = value[:, :, 0:4]
            pred_conf = value[:, :, 4:]

    boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
        boxes=tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)),
        scores=tf.reshape(
            pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])),
        max_output_size_per_class=50,
        max_total_size=50,
        iou_threshold=iou,
        score_threshold=score
    )

    # convert data to numpy arrays and slice out unused elements
    num_objects = valid_detections.numpy()[0]
    bboxes = boxes.numpy()[0]
    bboxes = bboxes[0:int(num_objects)]
    scores = scores.numpy()[0]
    scores = scores[0:int(num_objects)]
    classes = classes.numpy()[0]
    classes = classes[0:int(num_objects)]

    # format bounding boxes from normalized ymin, xmin, ymax, xmax ---> xmin, ymin, width, height
    original_h, original_w, _ = frame.shape
    bboxes = utils.format_boxes(bboxes, original_h, original_w)

    # store all predictions in one parameter for simplicity when calling functions
    pred_bbox = [bboxes, scores, classes, num_objects]

    # read in all class names from config
    class_names = utils.read_class_names(cfg.YOLO.CLASSES)

    # by default allow all classes in .names file
    allowed_classes = list(class_names.values())
    
    # custom allowed classes 
    allowed_classes = ['person','car','motorbike','bicycle','bus','truck']

    # loop through objects and use class index to get class name, allow only classes in allowed_classes list
    names = []
    deleted_indx = []
    for i in range(num_objects):
        class_indx = int(classes[i])
        class_name = class_names[class_indx]
        if class_name not in allowed_classes:
            deleted_indx.append(i)
        else:
            names.append(class_name)
    names = np.array(names)
    count = len(names)

    # delete detections that are not in allowed_classes
    bboxes = np.delete(bboxes, deleted_indx, axis=0)
    scores = np.delete(scores, deleted_indx, axis=0)

    # encode yolo detections and feed to tracker
    features = encoder(frame, bboxes)
    detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in zip(bboxes, scores, names, features)]

    #initialize color map
    cmap = plt.get_cmap('tab20b')
    colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

    # run non-maxima supression
    boxs = np.array([d.tlwh for d in detections])
    scores = np.array([d.confidence for d in detections])
    classes = np.array([d.class_name for d in detections])
    indices = preprocessing.non_max_suppression(boxs, classes, nms_max_overlap, scores)
    detections = [detections[i] for i in indices]       

    # Call the tracker
    tracker.predict()
    tracker.update(detections)

    # update tracks
    for track in tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue 
        bbox = track.to_tlbr()
        class_name = track.get_class()
        
    # draw bbox on screen
        color = colors[int(track.track_id) % len(colors)]
        color = [i * 255 for i in color]
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1]-30)), (int(bbox[0])+(len(class_name)+len(str(track.track_id)))*17, int(bbox[1])), color, -1)
        cv2.putText(frame, class_name + "-" + str(track.track_id),(int(bbox[0]), int(bbox[1]-10)),0, 0.75, (255,255,255),2)

        #if dominio(centroid_box(bbox),conflict_area[0]):
        check_count[str(track.track_id)] = str(passou_da_linha(centroid_box(bbox),conflict_area[0]))
        #ja tava comentado check_count[str(track.track_id)] = str(passou_da_linha_reta_horizontal(centroid_box(bbox),linha[1]))

        #if dominio(centroid_box(bbox),conflict_area[2]):
        check_count_2[str(track.track_id)] = str(passou_da_linha(centroid_box(bbox),conflict_area[2]))

        # Objeto dentro da área de interesse
        if dominio(centroid_box(bbox),conflict_area[1]):
            check_count_car[str(track.track_id)] = str(passou_da_linha(centroid_box(bbox),conflict_area[1]))

        if dominio(centroid_box(bbox),conflict_area[3]):
            check_count_car_2[str(track.track_id)] = str(passou_da_linha(centroid_box(bbox),conflict_area[3]))

        try: # Bloco do pedestres. A ideia é salvar o momento que o pedestre aparece no vídeo e verificar o momento que ele atravessa
            if (class_name == 'person') & (str(track.track_id) not in id_track_list): 
                inicio_linha1[str(track.track_id)] = check_count[str(track.track_id)]
                inicio_linha2[str(track.track_id)] = check_count_2[str(track.track_id)]
                #count_person += 1
                id_track_list.append(str(track.track_id))
                id_track_list = list(set(id_track_list))
                time_person_aux[str(track.track_id)] = frame_num
                x_person_inicial[str(track.track_id)] = centroid_box(bbox)[0]
                y_person_inicial[str(track.track_id)] = centroid_box(bbox)[1]

            #if (class_name == 'person') & (str(track.track_id) not in id_track_list): 
            #    count_person += 1
            #    id_track_list.append(str(track.track_id))
            #    id_track_list = list(set(id_track_list))
            #    time_person_aux[str(track.track_id)] = frame_num
            #    x_person_inicial[str(track.track_id)] = centroid_box(bbox)[0]
            #    y_person_inicial[str(track.track_id)] = centroid_box(bbox)[1]

            if (class_name == 'person') & (str(track.track_id) in id_track_list) & (check_count_2[str(track.track_id)] != check_count_2_aux[str(track.track_id)]) & (str(track.track_id) not in id_track_list_2):
                time_person[str(track.track_id)] = frame_num - time_person_aux[str(track.track_id)]
                id_track_list_2.append(str(track.track_id))
                id_track_list_2 = list(set(id_track_list_2))
                final_linha1[str(track.track_id)] = check_count[str(track.track_id)]
                final_linha2[str(track.track_id)] = check_count_2[str(track.track_id)]
                count_person += 1
                x_person_final[str(track.track_id)] = centroid_box(bbox)[0]
                y_person_final[str(track.track_id)] = centroid_box(bbox)[1]

            if (class_name == 'person') & (str(track.track_id) in id_track_list) & (check_count[str(track.track_id)] != check_count_aux[str(track.track_id)]) & (str(track.track_id) not in id_track_list_2):
                time_person[str(track.track_id)] = frame_num - time_person_aux[str(track.track_id)]
                id_track_list_2.append(str(track.track_id))
                id_track_list_2 = list(set(id_track_list_2))
                final_linha1[str(track.track_id)] = check_count[str(track.track_id)]
                final_linha2[str(track.track_id)] = check_count_2[str(track.track_id)]
                count_person += 1
                x_person_final[str(track.track_id)] = centroid_box(bbox)[0]
                y_person_final[str(track.track_id)] = centroid_box(bbox)[1]

            #print(check_count[str(track.track_id)])
            #print(check_count_aux[str(track.track_id)])
        except:
            pass
        

        try: # Bloco dos veículos, similar ao pedestre
            if (class_name != 'person') & (str(track.track_id) not in id_track_list) & (check_count_car[str(track.track_id)] != check_count_car_aux[str(track.track_id)]): 
                count_car += 1
                id_track_list.append(str(track.track_id))
                id_track_list = list(set(id_track_list))
                time_car_aux[str(track.track_id)] = frame_num
                car_class[str(track.track_id)] = class_name
                x_car_inicial[str(track.track_id)] = centroid_box(bbox)[0]
                y_car_inicial[str(track.track_id)] = centroid_box(bbox)[1]

            if (class_name != 'person') & (str(track.track_id) not in id_track_list) & (check_count_car_2[str(track.track_id)] != check_count_car_2_aux[str(track.track_id)]): 
                count_car += 1
                id_track_list.append(str(track.track_id))
                id_track_list = list(set(id_track_list))
                time_car_aux[str(track.track_id)] = frame_num
                car_class[str(track.track_id)] = class_name
                x_car_inicial[str(track.track_id)] = centroid_box(bbox)[0]
                y_car_inicial[str(track.track_id)] = centroid_box(bbox)[1]

            if (class_name != 'person') & (str(track.track_id) in id_track_list) & (check_count_car_2[str(track.track_id)] != check_count_car_2_aux[str(track.track_id)]) & (time_car_aux[str(track.track_id)] != frame_num):
                time_car[str(track.track_id)] = frame_num - time_car_aux[str(track.track_id)]
                x_car_final[str(track.track_id)] = centroid_box(bbox)[0]
                y_car_final[str(track.track_id)] = centroid_box(bbox)[1]

            if (class_name != 'person') & (str(track.track_id) in id_track_list) & (check_count_car[str(track.track_id)] != check_count_car_aux[str(track.track_id)]) & (time_car_aux[str(track.track_id)] != frame_num):
                time_car[str(track.track_id)] = frame_num - time_car_aux[str(track.track_id)]
                x_car_final[str(track.track_id)] = centroid_box(bbox)[0]
                y_car_final[str(track.track_id)] = centroid_box(bbox)[1]


        except:
            pass
        
        

    
    # if enable info flag then print details about each track
    #    if info:
            #print("Tracker ID: {}, Class: {},  BBox Coords (xmin, ymin, xmax, ymax): {}".format(str(track.track_id), class_name, (int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3]))))
    
    
    cv2.line(frame,(conflict_area[0][0],conflict_area[0][1]),(conflict_area[0][2],conflict_area[0][3]),(255,0,0),5) 
    cv2.line(frame,(conflict_area[1][0],conflict_area[1][1]),(conflict_area[1][2],conflict_area[1][3]),(255,0,0),5) 
    cv2.line(frame,(conflict_area[2][0],conflict_area[2][1]),(conflict_area[2][2],conflict_area[2][3]),(255,0,0),5) 
    cv2.line(frame,(conflict_area[3][0],conflict_area[3][1]),(conflict_area[3][2],conflict_area[3][3]),(255,0,0),5) 

    cv2.putText(frame, "Contador de pessoas: {}".format(count_person), (5, 35), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 255, 0), 2)
    cv2.putText(frame, "Contador de veiculos: {}".format(count_car), (5, 70), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 255, 0), 2)
    # calculate frames per second of running detections
    #fps = 1.0 / (time.time() - start_time)
    #print("FPS: %.2f" % fps)

    result = np.asarray(frame)
    result = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    #Progresso
    cont.update(1)
    
    
    # if output flag is set, save video file
    if saida:
        out.write(result)


    check_count_aux = check_count.copy()
    check_count_2_aux = check_count_2.copy()
    
    check_count_car_aux = check_count_car.copy()
    check_count_car_2_aux = check_count_car_2.copy()

    if cv2.waitKey(1) & 0xFF == ord('q'): break

print("Veiculos: %.2f" %  int(count_car))
print("Pedestres: %.2f" %  int(count_person))

print("Video in FPS: %.2f" %  int(vid.get(cv2.CAP_PROP_FPS)))
print("Video in n frames: %.2f" %  int(vid.get(cv2.CAP_PROP_FRAME_COUNT)))
print("Video in duration: %.2f" %  ((vid.get(cv2.CAP_PROP_FRAME_COUNT))/(vid.get(cv2.CAP_PROP_FPS))))

real_fps = frame_num / ((vid.get(cv2.CAP_PROP_FRAME_COUNT))/(vid.get(cv2.CAP_PROP_FPS)))
#out = cv2.VideoCapture(output)
#out.set(cv2.CAP_PROP_FPS, real_fps)

#print("Video real FPS: %.2f" %  int(real_fps))
#print("Video out FPS: %.2f" %  int(out.get(cv2.CAP_PROP_FPS)))
#print("Video out n frames: %.2f" %  int(out.get(cv2.CAP_PROP_FRAME_COUNT)))
#print("Video out duration: %.2f" %  ((out.get(cv2.CAP_PROP_FRAME_COUNT))/(out.get(cv2.CAP_PROP_FPS))))
#print(time.time())

time_start = time_person_aux
time_real = time_person

x_inicial = x_person_inicial
x_final = x_person_final
y_inicial = y_person_inicial
y_final = y_person_final

relatorio = pd.DataFrame(
    {
        'time_start': time_start,
        'time': time_real,
        'inicio_linha_1':inicio_linha1,
        'inicio_linha_2':inicio_linha2,
        'final_linha_1':final_linha1,
        'final_linha_2':final_linha2,
        'x_inicial': x_inicial,
        'y_inicial': y_inicial,
        'x_final': x_final,
        'y_final': y_final
    }
)
relatorio.to_csv("./outputs/"+ nome_arquivo_video +"/out.csv") # salvar os dados finais dos ped


time_start = time_car_aux
time_real = time_car

x_inicial = x_car_inicial
x_final = x_car_final
y_inicial = y_car_inicial
y_final = y_car_final

relatorio_2 = pd.DataFrame(
    {
        'time_start': time_start,
        'time': time_real,
        'classe': car_class,
        'inicio_linha_1':inicio_linha1,
        'inicio_linha_2':inicio_linha2,
        'final_linha_1':final_linha1,
        'final_linha_2':final_linha2,
        'x_inicial': x_inicial,
        'y_inicial': y_inicial,
        'x_final': x_final,
        'y_final': y_final
    }
)
relatorio_2.to_csv("./outputs/"+ nome_arquivo_video +"/out_car.csv") # salvar os dados finais dos veic


cont.close()
if False:
    vid = cv2.VideoCapture(output)

    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = real_fps
    codec = cv2.VideoWriter_fourcc(*output_format)
    out_put_path = output
    out2 = cv2.VideoWriter(out_put_path[:-4]+'_real_fps'+out_put_path[-4:], codec, fps, (width, height))


    while True:
        return_value, frame = vid.read()
        if return_value:
            frame = frame
        else:
            print('Video has ended or failed, try a different video format!')
            break
                # if output flag is set, save video file
        
        out2.write(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break



# Salvar csv

In [ ]:
fps = cv2.VideoCapture(output).get(cv2.CAP_PROP_FPS)
dados = pd.read_csv("./outputs/"+ nome_arquivo_video +"/out.csv")
dados_car = pd.read_csv("./outputs/"+ nome_arquivo_video +"/out_car.csv")
dados_car.head()

In [ ]:
dados.columns = ['id','time_start','time','inicio_linha_1', 	'inicio_linha_2', 	'final_linha_1', 	'final_linha_2','x_inicial',	'y_inicial',	'x_final',	'y_final']

In [ ]:
dados['time_start'] = dados['time_start'] / fps
dados['time'] = dados['time'] / fps

dados_car['time_start'] = dados_car['time_start'] / fps
dados_car['time'] = dados_car['time'] / fps

dados_completos = dados.copy()

In [ ]:
dados_completos.to_csv("./outputs/"+ nome_arquivo_video +"/atrasos.csv")
dados_car.to_csv("./outputs/"+ nome_arquivo_video +"/carros.csv")